<h2 id="Pembobotan-dan-Similarity"><strong>Preprocessing</strong></h2>

In [160]:
import nltk
import string
import re
import pandas as pd
import os
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import SnowballStemmer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from string import punctuation
from autocorrect import spell

snowball_stemmer = SnowballStemmer('english')
wordnet_lemmatizer = WordNetLemmatizer()

In [161]:
data = pd.read_csv(os.path.join("requirement.csv"))
df = data

In [162]:
df['req'].tail(5)

149    Fixed up the formatting and table of contents,...
150    The diagram designer is in charge of designing...
151    Ways to avoid scope creep are to develop chang...
152    Be careful of “scope creep” which are uncontro...
153    The major technology that you rely upon could ...
Name: req, dtype: object

In [163]:
def autospell(text):
    spells = [spell(w) for w in (nltk.word_tokenize(text))]
    return " ".join(spells)

def to_lower(text):
    return text.str.lower()

def remove_numbers(text):
    output = ''.join(c for c in text if not c.isdigit())
    return output

def remove_punct(text):
    return ''.join(c for c in text if c not in punctuation)

def remove_Tags(text):
    cleaned_text = re.sub('<[^<]+?>', '', text)
    return cleaned_text

def sentence_tokenize(text):
    sent_list = []
    for w in nltk.sent_tokenize(text):
        sent_list.append(w)
    return sent_list

def word_tokenize(text):
    return [w for sent in nltk.sent_tokenize(text) for w in nltk.word_tokenize(sent)]

def remove_stopwords(sentence):
    stop_words = stopwords.words('english')
    return ' '.join([w for w in nltk.word_tokenize(sentence) if not w in stop_words])

def stem(text):
    stemmed_word = [snowball_stemmer.stem(word) for sent in nltk.sent_tokenize(text)for word in nltk.word_tokenize(sent)]
    return " ".join(stemmed_word)

def lemmatize(text):
    lemmatized_word = [wordnet_lemmatizer.lemmatize(word)for sent in nltk.sent_tokenize(text)for word in nltk.word_tokenize(sent)]
    return " ".join(lemmatized_word)        

In [164]:
lower_text = to_lower(df['req'])
word_list=[]
for i in lower_text:
    i = remove_numbers(i)
    i = remove_punct(i)
    i = remove_Tags(i)
    i = remove_stopwords(i)
    i = lemmatize(i)
    word_list.append(i)

In [165]:
word_list

['user submit job associate deadline cost execution time',
 'user query cluster establish cost unit time submit job',
 'user monitor status submit job',
 'user cancel job submit',
 'user check credit balance',
 'user check usage history',
 'user check status node cluster',
 'user check usage pattern history cluster',
 'user check status submit job',
 'user check load node cluster',
 'user alters cost structure cluster',
 'user alters schedule policy cluster',
 'user take photo new student',
 'development process follow rational unified process rup approach',
 'sdms developed waterfall process model',
 'objective project develop meeting scheduler product organized routine allows u concentrate core element different phase easily efficiently',
 'web tier deal presentation logic page rendering',
 'eis tier responsible storing user profile ride advertisement',
 'runtime pattern applied',
 'development done extra application server',
 'application deployed application server working cluster 

In [166]:
dataprepp=pd.DataFrame(word_list)
dataprepp.to_csv('dataprep1.csv', index=False)

In [167]:
dataprepp.tail(5)

,0
149,fixed formatting table content added template ...
150,diagram designer charge designing diagram proj...
151,way avoid scope creep develop change control p...
152,careful “ scope creep ” uncontrolled change la...
153,major technology rely upon could possibly fail...


<h3>Konstruksi&nbsp;<strong>Word Embedding (skip-gram)</strong></h3>


In [168]:
import pandas as pd
from gensim.models import Word2Vec

def tokenization(lemmatized_word):
    #tokenization
    from nltk.tokenize import word_tokenize
    tokens = word_tokenize(lemmatized_word)
    return tokens

news = pd.read_csv('dataprep1.csv', sep=',', encoding='latin-1')
newss=[]

for i in range(100):
    news['0'][i] = tokenization(news['0'][i])
    newss.append(news['0'][i])

In [169]:
model = Word2Vec(newss, min_count=1, sg=1)
# summarize the loaded model
print(model)

Word2Vec(vocab=307, size=100, alpha=0.025)


In [170]:
# summarize vocabulary
words = list(model.wv.vocab)
print(words)

['user', 'submit', 'job', 'associate', 'deadline', 'cost', 'execution', 'time', 'query', 'cluster', 'establish', 'unit', 'monitor', 'status', 'cancel', 'check', 'credit', 'balance', 'usage', 'history', 'node', 'pattern', 'load', 'alters', 'structure', 'schedule', 'policy', 'take', 'photo', 'new', 'student', 'development', 'process', 'follow', 'rational', 'unified', 'rup', 'approach', 'sdms', 'developed', 'waterfall', 'model', 'objective', 'project', 'develop', 'meeting', 'scheduler', 'product', 'organized', 'routine', 'allows', 'u', 'concentrate', 'core', 'element', 'different', 'phase', 'easily', 'efficiently', 'web', 'tier', 'deal', 'presentation', 'logic', 'page', 'rendering', 'eis', 'responsible', 'storing', 'profile', 'ride', 'advertisement', 'runtime', 'applied', 'done', 'extra', 'application', 'server', 'deployed', 'working', 'future', 'ibm', 'running', 'jboss', 'database', 'mysql', 'redundancy', 'using', 'mirroring', 'raid', 'mode', 'workstation', 'run', 'linux', 'use', 'eclips

In [171]:
# save model
model.save('modeln.bin')
# load model
new_model = Word2Vec.load('modeln.bin')
print(new_model)

Word2Vec(vocab=307, size=100, alpha=0.025)


In [172]:
from sklearn.feature_extraction.text import CountVectorizer

def get_top_n_words(corpus, n=None):
    vec = CountVectorizer(stop_words = stopwords.words('english')).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_words(word_list, 20)
for word, freq in common_words:
    print(word, freq)
df2 = pd.DataFrame(common_words, columns = ['News' , 'count'])

role 54
system 44
user 32
item 30
report 27
per 26
document 21
staff 20
right 19
patron 18
server 16
administrator 16
student 14
project 14
library 14
application 13
hold 13
ability 13
design 12
record 12


In [173]:
result3 = model.most_similar('user', topn=5)
print(result3)

[('system', 0.32030031085014343), ('rendering', 0.3131704330444336), ('balance', 0.27029669284820557), ('development', 0.24968262016773224), ('view', 0.24678455293178558)]


C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [182]:
new_isomodel = Word2Vec.load('isomodel.bin')
print(new_isomodel)

Word2Vec(vocab=41, size=100, alpha=0.025)


In [183]:
results = new_isomodel.most_similar('administration', topn=5)
print(results)

[('environment', 0.19473305344581604), ('alert', 0.09444408118724823), ('virus', 0.07332561910152435), ('tester', 0.06715662032365799), ('develop', 0.0669461265206337)]


C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
